In this work, you will test various time series classification methods. You must choose **three** datasets from [the UCR/UEA time series repository](http://timeseriesclassification.com) and perform the tasks by evaluating the models on three selected datasets.


### Task 0: Preparation

You need to choose **three** datasets from the UCR/UEA time series repository. Please be careful since some UCR datasets can take a long time to be processed - You do not need to choose heavy datasets since it would slow your training/testing process. Use sktime's `load_UCR_UEA_dataset` function to perform. Please note that **you should use each dataset's original train/test splits** to train and report the test scores.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.callbacks import  EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2, L1

import numpy as np
from sklearn.preprocessing import LabelEncoder
from sktime.datasets import load_UCR_UEA_dataset

regularization_rate = 0.01

### Task 1: Time series classification using deep learning 1

Time series classification problems can be solved using networks like CNN, RNN, or FCN. You can even try to merge different networks. In this task, you must test your three chosen datasets on four other models.

Try to implement four different models: **1) Fully connected network at least with five dense layers, 2) One directional RNN, 3) 1D-CNN only, and 4) 1D-CNN +
GRU**. Note that you can always link the network to a fully connected layer to match the output size. You can freely construct any structure you want. **Report the average test scores of four models on three datasets you chose**. **`It would be four scores in total. Mark the best model in terms of the average test score`**. Briefly explain the structure you constructed. There is no definitive answer, and it is up to your own model. Here it would be best if you keep the following rules:

- When initially loading the dataset, use sktime's `load_UCR_UEA_dataset` function. This is for our grading purpose.
- You should use at least **two** Tensorflow callbacks when you fit your model. These can be built-in ones or your personalized callback.
- You should use Tensorflow's data API (`tf.data`) to manage your dataset and use `shuffle`, `batch,` and `prefetch` functions. This means that you need to convert the data format using the `from_tensor_slices` function. This also means that you need to create your own validation set. You are not limited to using any methods to do this, but you may also need to shuffle the dataset before (for that, you can use `np.random.permutation`). If you use Torch, explain how you implement the equivalent operations.
- You need to clearly report the **test accuracy** of the four models. Training and validation accuracy scores are not enough.
- You may need to deal with datasets of different sizes. In this case, it might be helpful to make a function to create a model that can receive different input sizes as a parameter.

In [2]:
def scheduler(epoch, lr):
    if epoch < 3:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
scheduler_callback = LearningRateScheduler(scheduler, verbose=1)

es_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=3,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

In [3]:
def train_and_evaluate_fully_connected(X_train , y_train, X_test, y_test):
    
    # Encode class values as integer
    encoder = LabelEncoder()
    encoder.fit(y_train)
    y_train = encoder.transform(y_train)
    y_test = encoder.transform(y_test)

    # convert integer to dummy variables (i.e. one hot encoded)

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    

    # number of classes
    num_classes = y_train.shape[-1]
    
    input_shape = X_train.shape[1:]

    num_classes =  y_train.shape[-1]
    input_shape= X_train.shape[1]
    input_shape = X_train.shape[1:]

    batch_size = 32


    # create tensorflow dataset from training data
    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    
    
    validation_split = 0.2
    num_validation_samples = int(len(X_train) * validation_split)
    train_ds = train_ds.skip(num_validation_samples)
    val_ds = train_ds.take(num_validation_samples)
    
    train_ds =(
        train_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    val_ds = (
        val_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    
    test_ds = (
        test_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    model = tf.keras.Sequential([
        layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(16, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate = 0.01), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_ds, epochs=20, validation_data=val_ds, callbacks=[es_callback, scheduler_callback])

    test_loss, test_acc = model.evaluate(test_ds)
    return test_acc

def train_and_evaluate_1D_CNN(X_train, y_train, X_test, y_test):

    # Encoding class values as Interger
    encoder = LabelEncoder()
    encoder.fit(y_train)
    y_train = encoder.transform(y_train)
    y_test = encoder.transform(y_test)

    # converting integers into dummy varibles (i.e. one hot encoded)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    num_classes = y_train.shape[-1]
    
    # Add new dimension to the input data to create 3D input shape
    X_train =np.expand_dims(X_train, axis=-1)
    X_test = np.expand_dims(X_test, axis=-1)

    # creating Tensorflow dataset
    train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=len(X_train))
    validation_split = 0.2
    num_validation_samples = int(len(X_train) * validation_split)
    train_ds = train_data.skip(num_validation_samples)
    val_ds = train_data.take(num_validation_samples)
    test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

    batch_size = 32
    train_ds =(
        train_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    val_ds = (
        val_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    
    test_ds = (
        test_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    
    input_shape = (X_train.shape[1], 1)
    model = tf.keras.Sequential([
        layers.Conv1D(filters=6, kernel_size=3, activation='relu', padding='same' , input_shape=input_shape, kernel_regularizer=L2(regularization_rate)),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(filters=16, kernel_size=3, activation='relu',padding='same', kernel_regularizer=L2(regularization_rate)),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(filters=32, kernel_size=3, activation='relu',padding='same', kernel_regularizer=L2(regularization_rate)),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(filters=64, kernel_size=3, activation='relu',padding='same', kernel_regularizer=L2(regularization_rate)),
        
        layers.MaxPooling1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_ds, epochs=10, batch_size=32, validation_data=val_ds, callbacks=[es_callback, scheduler_callback])

    # evaluating model
    test_loss, test_acc = model.evaluate(test_ds)

    return test_acc

In [4]:
def train_and_evaluate_1d_RNN(X_train, y_train, X_test, y_test):

    # encode target values as integer
    encoder = LabelEncoder()
    encoder.fit(y_train)
    y_train = encoder.transform(y_train)
    y_test = encoder.transform(y_test)

    # converting integers to dummy variables(i.e. one hot encoded)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    # numberof classes
    num_classes =  y_train.shape[-1]
    input_shape=(X_train.shape[1], X_train.shape[-1])

    # add a new dimention 
    X_train = np.expand_dims(X_train, axis=-1)
    X_test = np.expand_dims(X_test, axis=-1)
    input_shape=(X_train.shape[1], X_train.shape[-1])
    
    # creating Tensorflow dataset
    train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=len(X_train))
    validation_split = 0.2
    num_validation_samples = int(len(X_train) * validation_split)
    train_ds = train_data.skip(num_validation_samples)
    val_ds = train_data.take(num_validation_samples)
    test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

    batch_size = 32
    train_ds =(
        train_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    val_ds = (
        val_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    
    test_ds = (
        test_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    model = tf.keras.models.Sequential([
        layers.SimpleRNN(units=64, activation='relu', input_shape=input_shape),
        layers.Dense(units=64, activation='relu'),
        layers.Dense(units=128, activation='relu'),
        layers.Dense(units=num_classes, activation='softmax')
    ])

    
    # build model 
    model.build(input_shape=X_train.shape)
    model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(train_ds, epochs=10, batch_size=32, validation_data=val_ds, callbacks=[es_callback, scheduler_callback])

    #evaluate model
    test_loss, test_acc = model.evaluate(test_ds)
    return test_acc
    

In [5]:
def train_and_evaluate_1D_CNN_and_gru(X_train, y_train, X_test, y_test):

    # encode target values as integer
    encoder = LabelEncoder()
    encoder.fit(y_train)
    y_train = encoder.transform(y_train)
    y_test = encoder.transform(y_test)

    # converting integers to dummy variables(i.e. one hot encoded)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    # numberof classes
    num_classes =  y_train.shape[-1]

    # add a new dimention 
    X_train = np.expand_dims(X_train, axis=-1)
    X_test = np.expand_dims(X_test, axis=-1)

    # creating Tensorflow dataset
    train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=len(X_train))
    validation_split = 0.2
    num_validation_samples = int(len(X_train) * validation_split)
    train_ds = train_data.skip(num_validation_samples)
    val_ds = train_data.take(num_validation_samples)
    test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

    batch_size = 32
    train_ds =(
        train_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    val_ds = (
        val_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    
    test_ds = (
        test_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    model = tf.keras.models.Sequential([
        layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=X_train.shape[1:], kernel_regularizer=L2(regularization_rate)),
        layers.MaxPooling1D(pool_size=2),
        layers.GRU(32, return_sequences=True , kernel_regularizer=L2(regularization_rate)),
        layers.GRU(32 , kernel_regularizer=L2(regularization_rate)),
        layers.Dense(32, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
        
    ])

    model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_ds, epochs=10, batch_size=32, validation_data=val_ds, callbacks=[es_callback, scheduler_callback])

    test_loss, test_acc = model.evaluate(test_ds)

    return test_acc

In [6]:
datasets = ["BasicMotions", "ItalyPowerDemand", "AtrialFibrillation"]
def train_and_evaluate_on_datasets(dataset_names):
    fc_test_scores = []
    cnn_test_scores = []
    rnn_test_scores = []
    cnn_gru_test_scores = []
    for dataset_name in dataset_names:
        # Load dataset
        X_train, y_train = load_UCR_UEA_dataset(name=dataset_name, split='train',  return_type="numpy2D", extract_path=None)
        X_test, y_test = load_UCR_UEA_dataset(name=dataset_name, split='test',  return_type="numpy2D", extract_path=None)

        # Train and evaluate 1D CNN model
        cnn_test_acc = train_and_evaluate_1D_CNN(X_train, y_train, X_test, y_test)

        # Train and evaluate rnn model
        rnn_test_acc = train_and_evaluate_1d_RNN(X_train, y_train, X_test, y_test)

        # Train and evaluate fully connected network model
        fc_test_acc = train_and_evaluate_fully_connected(X_train, y_train, X_test, y_test)

        # Train and evaluate CNN-LSTM model
        cnn_gru_test_acc = train_and_evaluate_1D_CNN_and_gru(X_train, y_train, X_test, y_test)

        print(f'Test accuracy on {dataset_name} using 1D CNN:', cnn_test_acc)
        print(f'Test accuracy on {dataset_name} using one directional RNN:', rnn_test_acc)
        print(f'Test accuracy on {dataset_name} using fully connected network:', fc_test_acc)
        print(f'Test accuracy on {dataset_name} using CNN-GRU:', cnn_gru_test_acc)
        cnn_test_scores.append(cnn_test_acc)
        rnn_test_scores.append(rnn_test_acc)
        fc_test_scores.append(fc_test_acc)
        cnn_gru_test_scores.append(cnn_gru_test_acc)

    # Calculate the average test score for each model
    cnn_average_test_score = np.mean(cnn_test_scores)
    rnn_average_test_score = np.mean(rnn_test_scores)
    fc_average_test_score = np.mean(fc_test_scores)
    cnn_gru_average_test_score = np.mean(cnn_gru_test_scores)
    
    # Determine the best model
    models = ['1D CNN', 'RNN', 'Fully Connected Network', 'CNN-gru']
    scores = [cnn_average_test_score, rnn_average_test_score, fc_average_test_score, cnn_gru_average_test_score]
    best_model = models[np.argmax(scores)]
    best_score = np.max(scores)

    print(f'Average test accuracy for 1D CNN: {cnn_average_test_score}')
    print(f'Average test accuracy for RNN: {rnn_average_test_score}')
    print(f'Average test accuracy for fully connected network: {fc_average_test_score}')
    print(f'Average test accuracy for CNN-GRU: {cnn_gru_average_test_score}')
    print(f'The best model is {best_model} with an average test accuracy of {best_score}')

train_and_evaluate_on_datasets(datasets)


Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 2.0327 - accuracy: 0.3125 - val_loss: 3.2741 - val_accuracy: 0.1250 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 2/10
1/1 [==============================] - 0s 93ms/step - loss: 3.0488 - accuracy: 0.2812 - val_loss: 4.3741 - val_accuracy: 0.3750 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 3/10
1/1 [==============================] - 0s 72ms/step - loss: 5.7998 - accuracy: 0.1875 - val_loss: 2.4778 - val_accuracy: 0.2500 - lr: 0.0100

Epoch 4: LearningRateScheduler setting learning rate to 0.009048374369740486.
Epoch 4/10
1/1 [==============================] - 0s 82ms/step - loss: 2.2833 - accuracy: 0.4062 - val_loss: 1.5834 - val_accuracy: 0.6250 - lr: 0.0090

Epoch 5: LearningRateScheduler setting learning rate to 0.008187307976186275.

### Task 2: Time series classification using deep learning 2

There has been several neural network models dedicated to time series classification. Besides your own models that you developed in Lab 4, now you will develop such dedicated models by referring to some papers, and test if they indeed perform better than your rough models. There are two famous papers as follows:
 - [Convolutional neural networks for time series classification (2017)](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7870510)
 - [Time Series Classification from Scratch with Deep Neural Networks: A Strong Baseline (2017)](https://arxiv.org/abs/1611.06455)

First paper's idea is already implemented in sktime, with the name `CNNClassifier`. Second paper has three models and those are easy to develop using tensorflow. Now the task is to develop two models (MLP and FCN) in the second paper and test it together with `CNNClassifier`.

Use the same four datasets, and test sktime's `CNNClassifier` and MLP and FCN models you develop. Report test scores of three models (`CNNClassifier`, MLP, and FCN) on three datasets you chose. It would be nine scores in total. For MLP and FCN, you may need to satisfy the following requirement:

- You should use at least **two** Tensorflow callbacks when you fit your model. These can be built-in ones or your personalized callback. If you use Torch, explain how you implement the equivalent operations.
- You should run the model at least 10 epochs.
- You can use the same processed datasets in Task 1. For `CNNClassifier`, as you cannot use `tf.Data`, you may put the training set directly.
- For `CNNClassifier`, you can run it with the default parameters or reduce the number of epoch (default is 2000).
- Please use the predefined test dataset to report the test scores.

Note that the main purpose of this task is to check if you can develop a similar network structure with description. If the detail of the specific part (e.g., size of one layer or some custom parameters like epoch) is missing in the paper, you can set it on your own.


In [7]:
def mlp(X_train, y_train, X_test, y_test):

    # encoding class values
    encoder = LabelEncoder()
    encoder.fit(y_train)
    y_train = encoder.transform(y_train)
    y_test = encoder.transform(y_test)

    # converting into dummy data
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    num_classes = y_train.shape[-1]
    batch_size = 32

    # creating Tensorflow dataset
    train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=len(X_train))
    validation_split = 0.2
    num_validation_samples = int(len(X_train) * validation_split)
    train_ds = train_data.skip(num_validation_samples)
    val_ds = train_data.take(num_validation_samples)
    test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))


    batch_size = 32
    train_ds =(
        train_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    val_ds = (
        val_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    
    test_ds = (
        test_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    input_shape= input_shape = X_train.shape[1:]
    model = tf.keras.models.Sequential([
        layers.Flatten(input_shape=input_shape),
        layers.Dropout(0.1),

        layers.Dense(500, activation='relu'),
        layers.Dropout(0.2),

        layers.Dense(500, activation='relu'),
        layers.Dropout(0.2),

        layers.Dense(500, activation='relu'),
        layers.Dropout(0.3),

        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_ds, epochs=10, batch_size=32, validation_data=val_ds, callbacks=[es_callback, scheduler_callback])

    test_loss, test_acc = model.evaluate(test_ds)
    return test_acc

In [8]:
def train_and_evaluate_fcn(X_train, y_train, X_test, y_test):
    # encode class values as integers
    encoder = LabelEncoder()
    encoder.fit(y_train)
    y_train = encoder.transform(y_train)
    y_test = encoder.transform(y_test)

    # converting into dummy data
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    X_train = np.expand_dims(X_train, axis=-1)
    X_test = np.expand_dims(X_test, axis=-1)

    num_classes = y_train.shape[-1]
    batch_size = 32

    # creating Tensorflow dataset
    train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=len(X_train))
    validation_split = 0.2
    num_validation_samples = int(len(X_train) * validation_split)
    train_ds = train_data.skip(num_validation_samples)
    val_ds = train_data.take(num_validation_samples)
    test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

    batch_size = 32
    train_ds =(
        train_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    val_ds = (
        val_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    
    test_ds = (
        test_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    input_shape = X_train.shape[1:]
    model = tf.keras.models.Sequential([
        layers.Conv1D(filters=128, kernel_size=8, activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        
        layers.Conv1D(filters=256, kernel_size=5, activation='relu'),
        layers.BatchNormalization(),

        layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling1D(),
        layers.Dense(units=num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001) , loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_ds, epochs=10, batch_size=32, validation_data=val_ds, callbacks=[es_callback, scheduler_callback])
    test_loss, test_acc = model.evaluate(test_ds)
    return test_acc

In [9]:
from sktime.classification.deep_learning.cnn import CNNClassifier
from sklearn.metrics import accuracy_score

def train_and_evaluate_CNNclassifier(X_train, y_train, X_test, y_test):
    # enocoding the data
    encoder = LabelEncoder()
    encoder.fit(y_train)
    y_train = encoder.transform(y_train)
    y_test = encoder.transform(y_test)

    # creating cnn classifer with reduced epochs
    cnn_classifier = CNNClassifier(n_epochs=50)

    cnn_classifier.fit(X_train, y_train)

    # prediction 
    y_pred = cnn_classifier.predict(X_test)

    # evaluationg the model
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

In [10]:
mlp_scores = []
cnn_scores = []
fcn_scores = []
for dataset in datasets:
    X_train, y_train = load_UCR_UEA_dataset(name=dataset, split='train',  return_type="numpy2D", extract_path=None)
    X_test, y_test = load_UCR_UEA_dataset(name=dataset, split='test',  return_type="numpy2D", extract_path=None)

    # mlp classifier
    mlp_accuracy = mlp(X_train, y_train, X_test, y_test)
    mlp_scores.append(mlp_accuracy)

    # cnn
    cnn_accuracy = train_and_evaluate_CNNclassifier(X_train, y_train, X_test, y_test)
    cnn_scores.append(cnn_accuracy)

    # fcn
    fcn_accuracy = train_and_evaluate_fcn(X_train, y_train, X_test, y_test)
    fcn_scores.append(fcn_accuracy)
cnn_average = np.mean(cnn_scores)
fcn_average = np.mean(fcn_scores)
mlp_average = np.mean(mlp_scores)

for i in range(len(datasets)):
    print("Test accuracy of (CNNClassifier) on {} : {}".format(datasets[i], cnn_scores[i]))
    print("Test accuracy of (MLP) on {} : {}".format(datasets[i], mlp_scores[i]))
    print("Test accuracy of (FCn) on {} : {}".format(datasets[i], fcn_scores[i]))

# Report the rank of average accuracy scores
scores = [(cnn_average, "CNNClassifier"), (mlp_average, "MLPClassifier"), (fcn_average, "FCN model")]
scores.sort(reverse=True)  # Sort in descending order
for rank, (score, model) in enumerate(scores, 1):
    print(f"Rank {rank}: {model} - Average accuracy score: {score}")
    


Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 7.1471 - accuracy: 0.3125 - val_loss: 29.9361 - val_accuracy: 0.1250 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 2/10
1/1 [==============================] - 0s 128ms/step - loss: 20.9136 - accuracy: 0.1875 - val_loss: 164.5700 - val_accuracy: 0.2500 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 3/10
1/1 [==============================] - 0s 104ms/step - loss: 167.5061 - accuracy: 0.2188 - val_loss: 19.8928 - val_accuracy: 0.5000 - lr: 0.0100

Epoch 4: LearningRateScheduler setting learning rate to 0.009048374369740486.
Epoch 4/10
1/1 [==============================] - 0s 63ms/step - loss: 47.2762 - accuracy: 0.3750 - val_loss: 4.9289 - val_accuracy: 0.5000 - lr: 0.0090

Epoch 5: LearningRateScheduler setting learning rate to 0.008187307

### Task 3: Time series classification using deep learning 3

Next, you can try to further improve your model by selecting **two** of the following ideas:
- `Use Bi-Direction LSTM and CNN networks separately`, create two to three layers individually, and concatenate them. This means that until the third (or second) layer, you have two different networks handling the same dataset, and after that, you concatenate the output and finish with any FCN layer. Check [this post](https://stackoverflow.com/questions/59168306/how-to-combine-lstm-and-cnn-in-timeseries-classification) to get inspired.
- Apply any sktime's transformer (not attention transformer) first to the dataset and run any deep learning model you already developed in Tasks 2 and 3. In this case, you need to choose at least two transformers and apply them together.
- Train the model on multiple similar datasets and test it on one specific test set. Check if the model can be improved if it is trained on multiple datasets (at least five datasets). However, for this, you also need to choose the similar datasets based on their classification and motivate your choise in the report (UCR repository has a specific dataset type such as **AUDIO** or **MOTION**). You could try to crop or pad the time series if you would like to match the sizes.

Choose one model you want from the models you have developed in Tasks 1 and 2. Select one idea, try implementing it, and check if you can improve the performance. Note that you do not need to prove that the accuracy scores increase but must explain your trials. Report test scores on three datasets you chose.




**Answer:** I have implemented first and second approach seperately. and show the trials below.

In [11]:
from tensorflow.keras import layers, Input
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.pipeline import Pipeline

from sktime.transformations.series.difference import Differencer
from sktime.transformations.panel.tsfresh import TSFreshFeatureExtractor


from sktime.transformations.series.exponent import ExponentTransformer


def transform_x_by_transformer(X_train, X_test):
    exponent = ExponentTransformer(power=2)
    X_train  = exponent.fit_transform(X_train)
    X_test = exponent.fit_transform(X_test)
    
    differencer = Differencer()
    X_train = differencer.fit_transform(X_train)
    X_test = differencer.fit_transform(X_test)
    
    
    
    return X_train, X_test

def train_and_evaluate_combined_cnn_lstm(X_train, y_train, X_test, y_test):
    
    # Encoding
    encoder = LabelEncoder()
    encoder.fit(y_train)
    y_train = encoder.transform(y_train)
    y_test = encoder.transform(y_test)

    # converting into dummy variables(i.e one hot encoded)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)


    X_train = np.expand_dims(X_train, axis=-1)
    X_test = np.expand_dims(X_test, axis=-1)

    num_classes = y_train.shape[-1]
    batch_size = 32
    
    # creating Tensorflow dataset
    train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=len(X_train))
    validation_split = 0.2
    num_validation_samples = int(len(X_train) * validation_split)
    train_ds = train_data.skip(num_validation_samples)
    val_ds = train_data.take(num_validation_samples)
    test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

    train_ds =(
        train_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    val_ds = (
        val_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    
    test_ds = (
        test_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    input_shape=  (X_train.shape[1], X_train.shape[-1])

    input_layer = layers.Input(shape=input_shape)



    # LSTM Network functional model
    lstm = layers.Bidirectional(layers.LSTM(64, return_sequences=True, activation='tanh'))(input_layer)
    lstm = layers.Bidirectional(layers.LSTM(32, return_sequences=True, activation='tanh'))(lstm)
    lstm = layers.Bidirectional(layers.LSTM(16, return_sequences=True, activation='tanh'))(lstm)
    lstm = layers.Bidirectional(layers.LSTM(8, activation='tanh'))(lstm)

    # CNN Network functional model
    cnn = layers.Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=L2(regularization_rate))(input_layer)
    cnn = layers.MaxPooling1D(pool_size=2)(cnn)
    cnn = layers.Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=L2(regularization_rate))(cnn)
    cnn = layers.MaxPooling1D(pool_size=2)(cnn)
    cnn = layers.Conv1D(filters=16, kernel_size=3, activation='relu', kernel_regularizer=L2(regularization_rate))(cnn)
    cnn = layers.MaxPooling1D(pool_size=2)(cnn)
    cnn = layers.Flatten()(cnn)
    

    output = layers.concatenate([lstm, cnn])

    x = layers.Dense(64, activation='relu')(output)
    output_layer = layers.Dense(num_classes, activation='softmax')(x)

    # Create the model
    combined_model =tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
    combined_model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

    # model
    combined_model.fit(train_ds, epochs=10, batch_size=32, validation_data=val_ds, callbacks=[es_callback, scheduler_callback])

    # Evaluate model on test 
    test_loss, test_acc = combined_model.evaluate(test_ds)


    return test_acc
    


In [12]:
## second ideas
def transform_x_by_transformer(X_train, X_test):
    exponent = ExponentTransformer(power=2)
    X_train  = exponent.fit_transform(X_train)
    X_test = exponent.fit_transform(X_test)
    
    differencer = Differencer()
    X_train = differencer.fit_transform(X_train)
    X_test = differencer.fit_transform(X_test)
    
    return X_train, X_test


In [13]:
# training model on multiple datasets
datasets = ["BasicMotions", "ItalyPowerDemand", "AtrialFibrillation"]
test_accuracy = []
for data in datasets:
    X_train, y_train = load_UCR_UEA_dataset(name=data, split='train',  return_type="numpy2D", extract_path=None)
    X_test, y_test = load_UCR_UEA_dataset(name=data, split='test',  return_type="numpy2D", extract_path=None)
    accuracy = train_and_evaluate_combined_cnn_lstm(X_train, y_train, X_test, y_test)
    test_accuracy.append(accuracy)
for index, acc in enumerate(test_accuracy):
    print("test accuracy of {} dataset is {}".format(datasets[index], acc))


Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/10
1/1 [==============================] - 46s 46s/step - loss: 3.0290 - accuracy: 0.1875 - val_loss: 4.5521 - val_accuracy: 0.2500 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 4.0749 - accuracy: 0.2188 - val_loss: 3.6963 - val_accuracy: 0.3750 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 5.3325 - accuracy: 0.2188 - val_loss: 1.9962 - val_accuracy: 0.5000 - lr: 0.0100

Epoch 4: LearningRateScheduler setting learning rate to 0.009048374369740486.
Epoch 4/10
1/1 [==============================] - 2s 2s/step - loss: 1.9843 - accuracy: 0.4688 - val_loss: 1.5063 - val_accuracy: 0.8750 - lr: 0.0090

Epoch 5: LearningRateScheduler setting learning rate to 0.008187307976186275.
Epo

In [14]:
### second ideas
## first transform the data using sktime transformer then used a mlp model which implemented  in task 2.

datasets = ["BasicMotions", "ItalyPowerDemand", "AtrialFibrillation"]
test_accuracy = []
for data in datasets:
    X_train, y_train = load_UCR_UEA_dataset(name=data, split='train',  return_type="numpy2D", extract_path=None)
    X_test, y_test = load_UCR_UEA_dataset(name=data, split='test',  return_type="numpy2D", extract_path=None)
    X_train, X_test = transform_x_by_transformer(X_train, X_test)
    test_acc = mlp(X_train, y_train, X_test, y_test)
    test_accuracy.append(test_acc)
for index, accuracy in enumerate(test_accuracy):
    print("test accuracy of {} dataset is {}".format(datasets[index], accuracy))


Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 184.0564 - accuracy: 0.0938 - val_loss: 2336.8887 - val_accuracy: 0.3750 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 2/10
1/1 [==============================] - 0s 92ms/step - loss: 2427.7410 - accuracy: 0.2812 - val_loss: 5215.9141 - val_accuracy: 0.2500 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 3/10
1/1 [==============================] - 0s 90ms/step - loss: 3556.2427 - accuracy: 0.2188 - val_loss: 1439.1749 - val_accuracy: 0.5000 - lr: 0.0100

Epoch 4: LearningRateScheduler setting learning rate to 0.009048374369740486.
Epoch 4/10
1/1 [==============================] - 0s 119ms/step - loss: 988.1449 - accuracy: 0.4062 - val_loss: 143.5804 - val_accuracy: 0.5000 - lr: 0.0090

Epoch 5: LearningRateScheduler setting learning rate to

### Task 4: Time series classification using sktime

We can use `RandomizedSearch` to find optimal parameter options on pipelines. However, sktime's pipeline does not support scikit-learn's classifiers such as DecisionTree or RandomForest well. However, sometimes we would like to use the output of the sktime transformer (e.g., catch22) to train scikit-learn models such as RandomForest. Sktime supports this with `SklearnClassifierPipeline` to put a  scikit-learn classifier and sktime's transformer together and you need to implement it.

Pick one classifier from scikit-learn (that can be anything! e.g., Decision Tree or Logistic Regressor) and two transformers from sktime and create `SklearnClassifierPipeline.` As we tried in this lab, that can be **Rocket with RandomForest** or **Catch22 with DecisionTree**. Pick one parameter from each module (in total, three, one from the classifier and two from two transformers) and run a randomized search on the pipeline you define and report the test score of the best model found by the randomized Search. Compare your best score to the score from the same model with the default setting.

Task 4 involves a time-consuming process, so you can only choose **one dataset** to perform the task above. Also, note that you do not need to perform better by conducting a randomized search for this task (but still good to try!).

In [15]:
from sktime.transformations.panel.summarize import RandomIntervalFeatureExtractor
from sktime.transformations.panel.reduce import Tabularizer
from sktime.transformations.panel.pca import PCATransformer


from sktime.classification.compose import SklearnClassifierPipeline

from sktime.transformations.series.exponent import ExponentTransformer
from sktime.transformations.series.difference import Differencer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV

In [16]:


def transformer_and_model(X_train, y_train, X_test, y_test):
    exponentTransformer = ExponentTransformer()
    differencer = Differencer()
    clf = KNeighborsClassifier()

    # pipeline
    pipeline = SklearnClassifierPipeline(
        transformers=[
            ('exponentTransformer', exponentTransformer),
            ('differencer', differencer)
            ]
        ,
        classifier=clf
        )
    param_grid = {'exponentTransformer__power':[0.1,0.3,0.5, 0.7,0.9],
                  'differencer__lags': [1,2,3,4,5],
                  'clf__n_neighbors':[5,10,15,20] 
                 }
    model = RandomizedSearchCV(pipeline, param_grid,n_iter=10, cv=3, n_jobs=-1)
    model.fit(X_train, y_train)

    best_model = model.best_estimator_
    score = best_model.score(X_test, y_test)

    # best score and best estimator
    print("Best Score: {}".format( model.best_score_))
    print("best Estimator: {}".format( model.best_estimator_))

    print("test score of the best model is {}".format(score))
    

In [17]:
datasets = ["BasicMotions"]

for data in datasets:
    X_train, y_train=load_UCR_UEA_dataset(name=data, split='train', return_type="numpy2D", extract_path=None)
    X_test, y_test =load_UCR_UEA_dataset(name=data, split='test', return_type="numpy2D",   extract_path=None)

    print("Training Testing using Randomized searchcv hyperparameter tuning on {} dataset".format(data))
    transformer_and_model(X_train, y_train, X_test, y_test)

    print("Training Testing using default parameter on {} dataset".format(data))

    pipe = Pipeline([
        ("transform1", ExponentTransformer()),
        ("transform2", Differencer()),
        ("classifier", KNeighborsClassifier())
    ])
    pipe.fit(X_train, y_train)
    score = pipe.score(X_test, y_test)
    print("Test score of the default model is {}".format(score))
    

Training Testing using Randomized searchcv hyperparameter tuning on BasicMotions dataset


D:\Tensorflow-learn\tensorenv\lib\site-packages\sklearn\model_selection\_search.py:979: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Best Score: nan
best Estimator: SklearnClassifierPipeline(classifier=KNeighborsClassifier(),
                          transformers=[('exponentTransformer',
                                         ExponentTransformer(power=0.7)),
                                        ('differencer', Differencer(lags=2))])
test score of the best model is 0.325
Training Testing using default parameter on BasicMotions dataset
Test score of the default model is 0.25


### Task 5: Multivariate time series classification

Time series can be **multivariate**, which means there can be many values (= data points) describing one time point. In this task, you will use one **multivariate** dataset (**Eplipsy**) and try to run one deep learning model and one sktime model to see if those models work well on multivariate time series.

- Use Eplipsy dataset in the UCR/UEA repository.
- `Run two classifiers of your choice in sktime`, such as TapNet, Rocket, or MiniRocket, together with **the best tensorflow deep learning model from the previous tasks** on Eplipsy. You need to adjust the deep learning model's input layer to handle this multivariate dataset.
- Use sktime's `load_UCR_UEA_dataset` function to perform. You should use each dataset's original train/test splits.
- For the deep learning model, you should transform it using TensorFlow data API (`tf.data`) to manage your dataset and use `shuffle`, `batch`, and `prefetch` functions. This means that you need to create the validation set first. If you use Torch, explain how you implement the equivalent operations.
- For training, you need to run at least 10 epochs for your deep learning model. `For TapNet, you can keep the default parameter options`.
- Report the test scores of three models on the predefined test set.
- **Do the same task on one more chosen multivariate time series dataset.**

In [18]:
from sktime.classification.deep_learning.tapnet import TapNetClassifier
from sktime.classification.deep_learning.resnet import ResNetClassifier
from sktime.transformations.panel.rocket import Rocket
from sklearn.linear_model import RidgeClassifierCV

def test_resnet(X_train, y_train, X_test, y_test):
    # tapnet classifer
    # tapnet = TapNetClassifier(layers=(50, 30),  n_epochs=10, batch_size=32)
    restnet = ResNetClassifier(n_epochs=15, batch_size=32)
    restnet.fit(X_train, y_train)
    # evaluate the classifier on test data
    test_acc = restnet.score(X_test, y_test)
    print("ResNet Test Accuracy:", test_acc)


def test_rocket(X_train, y_train, X_test, y_test):
    #rocket classifier
    rocket = Rocket(random_state=42)
    rocket.fit(X_train)
    X_train_transform_rocket = rocket.transform(X_train)
    X_test_transform_rocket = rocket.transform(X_test)
    classifier_rocket = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
    classifier_rocket.fit(X_train_transform_rocket, y_train)
    test_acc = classifier_rocket.score(X_test_transform_rocket, y_test)
    print("Rocket test Accuracy:", test_acc)
    

In [19]:
## deep learning model
def deep_learning(X_train, y_train, X_test, y_test):
    # encoding the data
    encoder = LabelEncoder()
    encoder.fit(y_train)
    y_train = encoder.transform(y_train)
    y_test = encoder.transform(y_test)

    # converting into dummy data
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    X_train = np.expand_dims(X_train, axis=-1)
    X_test = np.expand_dims(X_test, axis=-1)

    num_classes = y_train.shape[-1]
    batch_size = 32
    # creating Tensorflow dataset
    train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=len(X_train))
    validation_split = 0.2
    num_validation_samples = int(len(X_train) * validation_split)
    train_ds = train_data.skip(num_validation_samples)
    val_ds = train_data.take(num_validation_samples)
    test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    

    batch_size = 32
    train_ds =(
        train_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    val_ds = (
        val_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    
    test_ds = (
        test_ds
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    input_shape = X_train.shape[1:]
    model = tf.keras.models.Sequential([
        # Example layers; adjust according to your dataset
        layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        layers.LSTM(32, return_sequences=True),
        layers.Flatten(),
        layers.Dense(num_classes, activation='sigmoid')
    ])

    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(train_ds, epochs=10, batch_size=32, validation_data=val_ds)

    # Evaluate model on test data
    test_loss, test_acc = model.evaluate(test_ds)
    print('Deep Learning Test accuracy:', test_acc)
    
    

In [20]:
X_train, y_train = load_UCR_UEA_dataset("Epilepsy", split="train", return_X_y=True)
X_test, y_test = load_UCR_UEA_dataset("Epilepsy", split="test", return_X_y=True)

test_resnet(X_train,y_train, X_test, y_test)
test_rocket(X_train, y_train, X_test, y_test)


5/5 [==============================] - 1s 84ms/step
ResNet Test Accuracy: 0.2753623188405797
Rocket test Accuracy: 0.9710144927536232


In [21]:
X_train, y_train = load_UCR_UEA_dataset("Epilepsy", split="train", return_type="numpy2D")
X_test, y_test = load_UCR_UEA_dataset("Epilepsy", split="test", return_type="numpy2D")
deep_learning(X_train, y_train, X_test, y_test)

Epoch 1/10
4/4 [==============================] - 10s 969ms/step - loss: 0.6083 - accuracy: 0.3273 - val_loss: 0.4829 - val_accuracy: 0.5556
Epoch 2/10
4/4 [==============================] - 2s 496ms/step - loss: 0.4769 - accuracy: 0.5455 - val_loss: 0.3602 - val_accuracy: 0.8519
Epoch 3/10
4/4 [==============================] - 2s 493ms/step - loss: 0.3736 - accuracy: 0.8091 - val_loss: 0.2881 - val_accuracy: 0.8889
Epoch 4/10
4/4 [==============================] - 2s 510ms/step - loss: 0.2922 - accuracy: 0.8909 - val_loss: 0.2258 - val_accuracy: 0.8889
Epoch 5/10
4/4 [==============================] - 2s 520ms/step - loss: 0.2171 - accuracy: 0.9000 - val_loss: 0.1833 - val_accuracy: 0.9259
Epoch 6/10
4/4 [==============================] - 2s 532ms/step - loss: 0.1436 - accuracy: 0.9273 - val_loss: 0.1572 - val_accuracy: 0.8889
Epoch 7/10
4/4 [==============================] - 2s 515ms/step - loss: 0.0866 - accuracy: 0.9818 - val_loss: 0.1262 - val_accuracy: 0.9259
Epoch 8/10
4/4 [===